Python用MetaTrader5のインポート

In [ ]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import time

ログイン情報を入力

In [ ]:
login_ID = 12345678 # ご自身のログインIDを入力
login_server = 'XMTrading-MT5' # ご自身のログインサーバーを入力
login_password = 'XXXXXXXXX' # ご自身のログインパスワードを入力

MT5に接続

In [ ]:
# ログイン情報で指定した取引口座でMetaTrader5に接続
if not mt5.initialize(login=login_ID, server=login_server,password=login_password):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

Input設定

In [ ]:
symbol = "GOLD" # 取引対象
first_lot = 0.01 # 初期ロット数
tp_factor = 2.0 # 利確係数
sl_factor = 1.5 # ストップロス係数
magic_number = 10001 # マジックナンバー 
slippage = 10 # スリッページ
bar_period = 30 # 取得する四本値の本数

各種情報取得

In [ ]:
point=mt5.symbol_info(symbol).point # 価格の最小単位

テクニカル指標の関数定義

In [ ]:
# ATR
def calculate_atr(data, period=14):
    data['tr'] = np.maximum((data['high'] - data['low']), 
                            np.maximum(abs(data['high'] - data['close'].shift()), abs(data['low'] - data['close'].shift())))
    data['atr'] = data['tr'].rolling(window=period).mean()
    return data

# MA
def calculate_ma(data, period=15, price_type='close'):
    data['ma'] = data[price_type].rolling(window=period).mean()
    return data

# RSI
def calculate_rsi(data, period=14, price_type='close'):
    delta = data[price_type].diff()
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    avg_gain = gains.rolling(window=period).mean()
    avg_loss = losses.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    data['rsi'] = 100 - (100 / (1 + rs))
    return data

売買判断フラグを設定する関数

In [ ]:
def generate_trade_flgs(data, symbol_tick):

    # テクニカル指標の計算
    data = calculate_ma(data)
    data = calculate_rsi(data)

    # 売買判断フラグ
    buy_flg = (symbol_tick.ask > data['ma'].iloc[-2]) & (data['rsi'].iloc[-2] < 30)
    sell_flg = (symbol_tick.bid < data['ma'].iloc[-2]) & (data['rsi'].iloc[-2] > 70)

    return buy_flg, sell_flg

ループ処理

In [ ]:
while True:
    symbol_tick = mt5.symbol_info_tick(symbol)

    # 15分足データを取得
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M15, 0, bar_period)
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')

    # ATRを計算
    data = calculate_atr(data)
    atr = data['atr'].iloc[-2]

    # 売買判断フラグを設定
    buy_flg, sell_flg = generate_trade_flgs(data, symbol_tick)

    # ポジション情報を取得
    buy_position = None
    sell_position = None
    positions = mt5.positions_get(group='*' + symbol + '*')

    for position in positions:
        order_type = position[5]

        if order_type == 0:
            buy_position = position
        elif order_type == 1:
            sell_position = position


    # buyエントリー
    if buy_flg and buy_position is None:
        tp = round((symbol_tick.ask + atr * tp_factor) / point) * point
        sl = round((symbol_tick.ask - atr * sl_factor) / point) * point

        request = {
            'symbol': symbol,
            'action': mt5.TRADE_ACTION_DEAL,
            'type': mt5.ORDER_TYPE_BUY,
            'volume': first_lot,
            'price': symbol_tick.ask,
            'sl': sl,
            'tp': tp,
            'deviation': slippage,
            'magic': magic_number,
            'type_time': mt5.ORDER_TIME_GTC,
            'type_filling': mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)
    
    # sellエントリー
    if sell_flg and sell_position is None:
        tp = round((symbol_tick.bid - atr * tp_factor) / point) * point
        sl = round((symbol_tick.bid + atr * sl_factor) / point) * point

        request = {
            'symbol': symbol,
            'action': mt5.TRADE_ACTION_DEAL,
            'type': mt5.ORDER_TYPE_SELL,
            'volume': first_lot,
            'price': symbol_tick.bid,
            'sl': sl,
            'tp': tp,
            'deviation': slippage,
            'magic': magic_number,
            'type_time': mt5.ORDER_TIME_GTC,
            'type_filling': mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)


    time.sleep(60)  # 1分間隔でループを実行
